In [4]:
from pathlib import Path
dp = Path("/Users/chrismutel/Library/Application Support/EcoinventInterface/cache/ecoinvent 3.6_consequential_ecoSpold02/MasterData")
assert dp.is_dir()

In [2]:
from bw2io.importers.ecospold2_biosphere import *

In [8]:
class Ecospold2BiosphereImporter(LCIImporter):
    format = "Ecoinvent XML"

    def __init__(self, name="biosphere3", version="3.9"):
        self.db_name = name
        self.strategies = [
            normalize_units,
            drop_unspecified_subcategories,
            ensure_categories_are_tuples,
        ]

    def extract(self):
        def extract_flow_data(o):
            ds = {
                "category": o.compartment.compartment.text,
                "subcategory": o.compartment.subcompartment.text,
                "uuid": o.get("id"),
                "CAS number": o.get("casNumber"),
                "name": o.name.text,
                "unit": o.unitName.text,
            }
            return ds

        fp = dp / "ElementaryExchanges.xml"
        root = objectify.parse(open(fp, encoding="utf-8")).getroot()
        flow_data = recursive_str_to_unicode(
            [extract_flow_data(ds) for ds in root.iterchildren()]
        )

        # previous = os.path.join(lci_dirpath, "previous elementary flows.json")
        # return flow_data + json.load(open(previous))
        return flow_data

    
data = Ecospold2BiosphereImporter().extract()

In [10]:
import pandas as pd
pd.DataFrame(data).to_excel("biosphere 3.6.xlsx")